## Setup

Before start this notebook, you need to:
1. Install ImJoy Jupyter Extension by run `pip install imjoy-jupyter-extension`, restart Jupyter notebook, and make sure you see an ImJoy icon in the toolbar in opened notebooks.
1. Download the lastest version of [micro-manager 2.0](https://micro-manager.org/wiki/Micro-Manager_Nightly_Builds)
1. Install pymmcore using `pip install pymmcore`

In [ ]:
!pip install -U pymmcore imjoy

## Set the path to your micromanager installation and config file

In [ ]:
import os
MM_DIR = "./mmcore"
MM_CONFIG_FILE = os.path.join(MM_DIR, "MMConfig_demo.cfg")

# Acquire images with pymmcore

In [ ]:
import numpy as np
from imjoy import api
import pymmcore
import os.path

mmc = pymmcore.CMMCore()
mmc.setDeviceAdapterSearchPaths([MM_DIR])
mmc.loadSystemConfiguration(MM_CONFIG_FILE)
mmc.snapImage()
mmc.getImage()

# Display image with an ImJoy window plugin

In the following example, we will wrap the `MyMicroscope` class as an ImJoy plugin (by adding a `run` function). In the `run` function, we create a viewer by using the `itk-vtk-viewer` hosted on https://oeway.github.io/itk-vtk-viewer/

In [ ]:
import numpy as np
from imjoy import api
import pymmcore
import os.path

mmc = pymmcore.CMMCore()
mmc.setDeviceAdapterSearchPaths([MM_DIR])
mmc.loadSystemConfiguration(MM_CONFIG_FILE)
mmcore = mmc

class MyMicroscope():
    def setup(self):
        self._core = mmcore
        self._core.setExposure(float(10))
        exposure = self._core.getExposure()
        api.showMessage('MMcore loaded, exposure: ' + str(exposure))
    
    def snapImage(self):
        self._core.snapImage()
        image_array = self._core.getImage()
        image_array = (image_array/image_array.max()*255).astype('uint8')
        return image_array

    async def run(self, ctx):
        viewer = await api.showDialog(type="itk-vtk-viewer",
                                      src="https://oeway.github.io/itk-vtk-viewer/")
        api.showMessage('Acquiring 100 images')
        for i in range(100):
            await viewer.imshow(self.snapImage())
        api.showMessage('Done.')
        
api.export(MyMicroscope())

# Build a custom UI plugin for microscope control

Since the above image viewer doesn't give much flexiblity, we can build a custom UI plugin to add some custom UI for controlling the microscope.

The following example uses [Vue.js](https://vuejs.org/) and [Buefy](https://buefy.org/) to add buttons and controls. In addition, we introduce the [itk-vtk-viewer](https://kitware.github.io/itk-vtk-viewer/docs/embeddedViewer.html) in this UI plugin for displaying the numpy array image.

In [ ]:
## ImJoy Plugin
from IPython.display import HTML
my_plugin_source = HTML('''
<docs lang="markdown">
[TODO: write documentation for this plugin.]
</docs>

<config lang="json">
{
  "name": "PycroCam",
  "type": "window",
  "tags": [],
  "ui": "",
  "version": "0.1.2",
  "api_version": "0.1.8",
  "description": "A microscopy control plugin built on top of PycroManager",
  "icon": "extension",
  "inputs": null,
  "outputs": null,
  "env": "",
  "requirements": [
      "https://cdnjs.cloudflare.com/ajax/libs/vue/2.5.22/vue.min.js",
      "https://unpkg.com/buefy/dist/buefy.min.css",
      "https://unpkg.com/buefy/dist/buefy.min.js",
      "https://cdn.materialdesignicons.com/5.3.45/css/materialdesignicons.min.css",
      "https://oeway.github.io/itk-vtk-viewer/itkVtkViewerCDN.js"
  ],
  "dependencies": [],
  "defaults": {"w": 20, "h": 10},
  "runnable": true
}
</config>

<script lang="javascript">

api.registerCodec({
  name: 'itkimage',
  decoder: itkVtkViewer.utils.convertToItkImage
})

api.registerCodec({
  name: 'ndarray',
  decoder: itkVtkViewer.utils.ndarrayToItkImage
})

const app = new Vue({
  el: '#app',
  data: {
    mmcore: null,
    viewer: null,
    liveRunning: false,
    binning: 1,
    exposure: 100,
    cameraDevice: null,
  },
  methods: {
    async init(mmcore){
        this.mmcore = mmcore;
        this.cameraDevice = await this.mmcore.getCameraDevice()
        this.exposure = await this.mmcore.getExposure()
        this.binning = await this.mmcore.getProperty(this.cameraDevice, 'Binning')
        this.updateImage()
    },
    async createViewer(imageData, el) {
        imageData = itkVtkViewer.utils.vtkITKHelper.convertItkToVtkImage(imageData)
        // clear container
        el.innerHTML = "";
        const toolbar = document.createElement('div')
        el.appendChild(toolbar)
        const view = document.createElement('div')
        el.appendChild(view)
        const dims = imageData.getDimensions()
        const is2D = dims.length === 2 || (dims.length === 3 && dims[2] === 1)
        const viewer = itkVtkViewer.createViewer(view, {
          image: imageData,
          pointSets: null,
          geometries: null,
          use2D: is2D,
          rotate: false,
          uiContainer: toolbar
        })
        return viewer
    },
    async updateImage(){
      if(!this.mmcore){
          api.alert('No mmcore api provided.')
          return
      }
      let img;
      if(this.liveRunning){
          img = await this.mmcore.getImage()
      }
      else{
          img = await this.mmcore.snapImage()
      }
    
      if(!this.viewer){
        const elm = document.getElementById('viewer')
        this.viewer = await this.createViewer(img, elm)
      }
      else{
        this.viewer.setImage(itkVtkViewer.utils.vtkITKHelper.convertItkToVtkImage(img))
      }
      if(this.liveRunning){
          this.updateImage()
      }
    },
    async toggleLive(){
      this.liveRunning = !this.liveRunning
      if(this.liveRunning){
          this.mmcore.startContinuousSequenceAcquisition(0)
          this.updateImage()
      }
      else{
          this.mmcore.stopSequenceAcquisition()
      }
    },
    async showDevicePropertyBrowser(){
        const browser = await api.showDialog({src: "https://gist.github.com/oeway/d07f14fca4d9ab1febbde200d0369bf2", data:{'mmcore': this.mmcore}})
        browser.on('property_changed', (item)=>{
            if(item.device === this.cameraDevice && item.prop === 'Exposure'){
                this.exposure = item.value
            }
            else if(item.device === this.cameraDevice && item.prop === 'Binning'){
                this.binning = item.value
            }
        })
    }
  }
})

class ImJoyPlugin {
  async setup() {

  }

  async run(ctx) {
    await app.init(ctx.data.mmcore)
  }
}

api.export(new ImJoyPlugin())
</script>

<window lang="html">
  <div id="app">
        <section>
            <div class="buttons">
                <b-button type="is-primary" @click="updateImage()" :loading="liveRunning">Snap</b-button>
                <b-button :type="liveRunning?'is-danger': 'is-primary is-light'"
                          @click="toggleLive()">{{liveRunning?'Stop Live': 'Start Live'}}</b-button>
                &nbsp;Exposure: <b-field>
                    <b-input placeholder="Exposure"
                        size="is-small"
                        @input="mmcore.setExposure(parseFloat(exposure))"
                        v-model="exposure"
                        type="number"
                        min="0"
                        max="10000">
                    </b-input>
                </b-field>
                &nbsp;Binning: <b-field>
                    <b-select placeholder="Binning"
                              size="is-small"
                              @input="mmcore.setProperty(cameraDevice, 'Binning', binning)"
                              v-model="binning">
                        <option :value="1">1</option>
                        <option :value="2">2</option>
                        <option :value="4">4</option>
                        <option :value="8">8</option>
                        <option :value="16">16</option>
                    </b-select>
                </b-field>
                &nbsp;&nbsp;<b-button size="is-small" @click="showDevicePropertyBrowser()">Device Properties</b-button>
            </div>
            

        </section>
        <section id="viewer-section">
            <div id="viewer" class="viewer-container">

            </div>
      </section>
  </div>
</window>

<style lang="css">
#app{
    margin-top: 2px;
}
#viewer-section{
  display: table;
  margin-top: 5px;
  max-width: calc( 100vh -  36px );
}
.viewer-container{
  display: table-cell;
  position: relative;
}
    
.field {
    margin-bottom: .2rem!important;
}
</style>
''')

# Use the UI plugin with PycroManager

In [ ]:
import re

class DotDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

def to_camel_case(snake_str):
    components = snake_str.split('_')
    # We capitalize the first letter of each component except the first one
    # with the 'title' method and join them together.
    return components[0] + ''.join(x.title() for x in components[1:])


class MMCoreWrapper():
    '''
    make a wrapper to mmcore so the function calls are more pythonic
    meaning, all the camel case are converted into snake case
    for example: mmc.setExposure are converted into mmc.set_exposure
    '''
    def __init__(self, obj):
        self._wrapped_obj = obj

    def get_tagged_image(self):
        img = self.get_image()
        return DotDict({"pix": img, "tags": {"Height": img.shape[0], "Width": img.shape[1]}})

    def __getattr__(self, attr):
        if attr in self.__dict__:
            return getattr(self, attr)
        attr = to_camel_case(attr)
        return getattr(self._wrapped_obj, attr)

In [ ]:
import time
from imjoy import api
import numpy as np
import pymmcore
import os.path

mmc = pymmcore.CMMCore()
mmc.setDeviceAdapterSearchPaths([MM_DIR])
mmc.loadSystemConfiguration(MM_CONFIG_FILE)
mmcore = MMCoreWrapper(mmc)


class MyMicroscope():
    async def setup(self):
        self._core = mmcore
        exposure = self._core.get_exposure()
        api.showMessage('MMcore loaded, exposure: ' + str(exposure))

    def snap_image(self):
        if self._core.is_sequence_running():
            self._core.stop_sequence_acquisition()
        self._core.snap_image()
        tagged_image = self._core.get_tagged_image()
        image_array = np.reshape(tagged_image.pix, newshape=[-1, tagged_image.tags['Height'], tagged_image.tags['Width']])
        image_array = (image_array/image_array.max()*255).astype('uint8')
        return image_array
    
    def get_image(self):
        # we can also check remaining with getRemainingImageCount()
        tagged_image = self._core.get_tagged_image()
        image_array = np.reshape(tagged_image.pix, newshape=[-1, tagged_image.tags['Height'], tagged_image.tags['Width']])
        image_array = (image_array/image_array.max()*255).astype('uint8')
        return image_array

    def get_device_properties(self):
        devices = core.get_loaded_devices()
        if not isinstance(devices, tuple):
            devices = [devices.get(i) for i in range(devices.size())]

        device_items = []
        for device in devices:
            props = core.get_device_property_names(device)
            if not isinstance(props, tuple):
                props = [props.get(i) for i in range(props.size())]
            property_items = []
            for prop in props:
                value = core.get_property(device, prop)
                is_read_only = core.is_property_read_only(device, prop)
                if core.has_property_limits(device, prop):
                    lower = core.get_property_lower_limit(device, prop)
                    upper = core.get_property_upper_limit(device, prop)
                    allowed = {"type": "range", "min": lower, "max": upper, "readOnly": is_read_only}
                else:
                    allowed = core.get_allowed_property_values(device, prop)
                    if not isinstance(allowed, tuple):
                        allowed = [allowed.get(i) for i in range(allowed.size())]
                    allowed = {"type": "enum", "options": allowed, "readOnly": is_read_only}
                property_items.append({"device": device, "name": prop, "value": value, "allowed": allowed})
                # print('===>', device, prop, value, allowed)
            if len(property_items) > 0:
                device_items.append({"name": device, "value": "{} properties".format(len(props)), "items": property_items})
        return device_items

    async def run(self, ctx):
        mmcore_api = {
            "_rintf": True,
            "snapImage": self.snap_image,
            "getImage": self.get_image,
            "getDeviceProperties": self.get_device_properties,
            "getCameraDevice": self._core.get_camera_device,
            "setCameraDevice": self._core.set_camera_device,
            "startContinuousSequenceAcquisition": self._core.start_continuous_sequence_acquisition,
            "stopSequenceAcquisition": self._core.stop_sequence_acquisition,
            "setExposure": self._core.set_exposure,
            "getExposure": self._core.get_exposure,
            "setProperty": self._core.set_property,
            "getProperty": self._core.get_property
        }
        viewer = await api.createWindow(src=my_plugin_source, data={'mmcore': mmcore_api})

api.export(MyMicroscope())